In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


# Предобработка

In [3]:
# создадим словарь для ответов
answers = {}

# изменим формат вывода значений float
pd.options.display.float_format = '{:.1f}'.format

# изменим формат Date
data.release_date = pd.to_datetime(data.release_date)

# добавляем колонку с вычисленной прибылью (= сборы - бюджет)
data['profit'] = data.revenue - data.budget

# создаем отдельный DataFrame с жанрами фильмов
genres = data.genres.str.split('|', expand = True)

# создаем отдельный DataFrame с режиссерами фильмов
directors = data.director.str.split('|', expand = True)

# создаем отдельный DataFrame с актерами
casts = data.cast.str.split('|', expand = True)

# создаем отдельный DataFrame с производственными компаниями
companies = data.production_companies.str.split('|', expand = True)

# 1. У какого фильма из списка самый большой бюджет?

In [4]:
answers['1'] = data[data.budget == data.budget.max()].original_title
display(answers['1'])

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

Вариант 2

In [5]:
max_budget = data.budget.max()
data.query('budget == @max_budget').original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
answers['2'] = data[data.runtime == data.runtime.max()].original_title
display(answers['2'])

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
answers['3'] = data[data.runtime == data.runtime.min()].original_title
display(answers['3'])

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = round(data.runtime.mean(), 1)
display(answers['4'])

109.7

# 5. Каково медианное значение длительности фильмов?

In [9]:
answers['5'] = data.runtime.median()
display(answers['5'])

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
answers['6'] = data[data.profit == data.profit.max()].original_title
display(answers['6'])

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [11]:
answers['7'] = data[data.profit == data.profit.min()].original_title
display(answers['7'])

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
answers['8'] = data.query('profit > 0').profit.count()
display(answers['8'])

1478

#### *Вариант 2*

In [13]:
data[data.revenue > data.budget].profit.count()

1478

#### *Вариант 3*

In [14]:
len(data[data.revenue > data.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [15]:
answers['9'] = data[data.revenue == data[data.release_year == 2008].revenue.max()].original_title
display(answers['9'])

599    The Dark Knight
Name: original_title, dtype: object

#### *Вариант 2*

In [16]:
max_revenue_2008 = data[data.release_year == 2008].revenue.max()
data.query('revenue == @max_revenue_2008').original_title

599    The Dark Knight
Name: original_title, dtype: object

#### *Вариант 3*

In [17]:
data.\
    query('release_year == 2008')[['original_title', 'revenue']].\
    sort_values(by='revenue', ascending=False).\
    head(1)

,original_title,revenue
599,The Dark Knight,1001921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [18]:
answers['10'] = data[data.profit == data[(data.release_year >= 2012)&(data.release_year <= 2014)].
                     profit.min()].original_title
display(answers['10'])

1245    The Lone Ranger
Name: original_title, dtype: object

#### *Вариант 2*

In [19]:
max_loss = data.query('2014 >= release_year >= 2012').profit.min()
data.query('profit == @max_loss').original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [20]:
# решение через numpy
answers['11'] = pd.value_counts(genres.values.ravel()).idxmax()
display(answers['11'])

'Drama'

#### *Вариант 2*

In [21]:
# решение через collection Counter
Counter(data.genres.str.cat(sep = '|').split('|')).most_common(1)

[('Drama', 782)]

#### *Вариант 3*

In [22]:
pd.DataFrame(data.genres.str.cat(sep = '|').split('|'))[0].value_counts().idxmax()

'Drama'

#### *Вариант 4*

In [23]:
data.genres.str.split('|').explode().value_counts().idxmax()

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [24]:
data.query('profit > 0').genres.str.split('|').explode().value_counts().idxmax()

'Drama'

#### *Вариант 2*

In [25]:
sorted_genres = data.query('profit > 0').genres.str.split('|', expand = True)
pd.value_counts(sorted_genres.values.ravel()).idxmax()

'Drama'

#### *Вариант 3*

In [26]:
answers['12'] = genres.\
                iloc[data.query('profit > 0').index].\
                melt(value_name = 'genres').\
                genres.value_counts().\
                idxmax()
display(answers['12'])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [27]:
director_list = pd.DataFrame(data.director.str.split('|').explode())
director_list['revenue'] = data.revenue
answers['13'] = director_list.groupby('director').revenue.sum().idxmax()
display(answers['13'])

'Peter Jackson'

#### *Вариант 2*

In [28]:
directors['revenue'] = data.revenue # добавляем колонку со сборами

s = directors.\
    melt(id_vars = 'revenue', value_name = 'director').\
    groupby('director').\
    revenue.sum().\
    idxmax()

directors = directors.drop('revenue', axis = 1) # возвращаем к исходному виду

display(s)

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [29]:
answers['14'] = data[data.genres.str.contains('Action', na = False)].\
                director.str.split('|').explode().\
                value_counts().idxmax()
display(answers['14'])

'Robert Rodriguez'

#### *Вариант 2*

In [30]:
directors[data.genres.str.contains('Action', na = False)].\
    melt(value_name = 'director').\
    director.value_counts().\
    idxmax()

'Robert Rodriguez'

#### *Вариант 3*

In [31]:
Counter(data[data.genres.str.contains('Action', na = False)].\
        director.str.cat(sep = '|').split('|')).\
        most_common(1)

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [32]:
cast_list = pd.DataFrame(data.cast.str.split('|').explode())
cast_list['revenue'] = data.revenue
cast_list['release_year'] = data.release_year
cast_list.query('release_year == 2012').groupby('cast').revenue.sum().idxmax()

# cast_list.iloc[data.query('release_year == 2012').index].groupby('cast').revenue.sum().idxmax()
# не пойму почему не работает 

'Chris Hemsworth'

In [33]:
cast_list_2012 = pd.DataFrame(data.query('release_year == 2012').cast.str.split('|').explode())
cast_list_2012['revenue'] = data.revenue
cast_list_2012.groupby('cast').revenue.sum().idxmax()

'Chris Hemsworth'

In [34]:
casts['revenue'] = data.revenue # добавляем колонку со сборами

answers['15'] = casts.\
                iloc[data.query('release_year == 2012').index].\
                melt(id_vars = 'revenue', value_name = 'cast').\
                groupby('cast').\
                revenue.sum().\
                idxmax()

casts = casts.drop('revenue', axis = 1) # возвращаем к исходному виду

display(answers['15'])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [35]:
answers['16'] = data[data.budget >= data.budget.mean()].\
                cast.str.split('|').explode().\
                value_counts().\
                idxmax()
display(answers['16'])

'Matt Damon'

In [36]:
casts.iloc[data.query('budget > budget.mean()').index].\
melt(value_name = 'cast').\
cast.value_counts().\
idxmax()

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [37]:
answers['17'] = data[data.cast.str.contains('Nicolas Cage', na = False)].\
                genres.str.split('|').explode().\
                value_counts().\
                idxmax()
display(answers['17'])

'Action'

In [38]:
genres[data.cast.str.contains('Nicolas Cage', na = False)].\
melt(value_name = 'gener').\
gener.value_counts().\
idxmax()

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [39]:
PP_max_loss = data[data.production_companies.str.contains('Paramount Pictures', na = False)].profit.min()
answers['18'] = data.query('profit == @PP_max_loss').original_title
display(answers['18'])

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [40]:
answers['19'] = data.\
                groupby('release_year').\
                revenue.agg(['sum']).\
                sort_values(by = 'sum', ascending = False).\
                idxmax()
display(answers['19'])

sum    2015
dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [41]:
answers['20'] = data[data.production_companies.str.contains('Warner Bros', na = False)].\
                groupby('release_year').\
                profit.agg(['sum']).\
                sort_values(by = 'sum', ascending = False).\
                idxmax()
display(answers['20'])

sum    2014
dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [42]:
answers['21'] = data.groupby(data.release_date.dt.month).\
                imdb_id.count().\
                sort_values(ascending = False).\
                idxmax()
display(answers['21'])

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [43]:
answers['22'] = data[data.release_date.dt.month.isin([6, 7, 8])].imdb_id.count()
display(answers['22'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [44]:
'''
ответ основан на комментарии ментора в Slack:
"Для каждого режиссера вам нужно посчитать,
сколько фильмов он выпустил зимой и найти того,
у которого это значение максимальное"
'''
answers['23'] = directors[data.release_date.dt.month.isin([1, 2, 12])].\
                melt(value_name = 'director').\
                director.value_counts().\
                idxmax()

display(answers['23'])

'Peter Jackson'

In [45]:
'''
хотя, исходя из формулировки вопроса,
самым продуктивным временем года у режиссера будет считаться тот период
в котором он выпустил наибольшее количество фильмов по сравнению с другими периодами
'''
winter_period = directors[data.release_date.dt.month.isin([12, 1, 2])].\
                melt(value_name = 'director').director.value_counts()

spring_period = directors[~data.release_date.dt.month.isin([3, 4, 5])].\
                melt(value_name = 'director').director.value_counts()

summer_period = directors[~data.release_date.dt.month.isin([6, 7, 8])].\
                melt(value_name = 'director').director.value_counts()

autumn_period = directors[~data.release_date.dt.month.isin([9, 10, 11])].\
                melt(value_name = 'director').director.value_counts()

total_df = pd.concat([winter_period, spring_period, summer_period, autumn_period],
                     axis = 1, keys = ['winter', 'spring', 'summer', 'autumn'])

display(total_df.head())

display(total_df.query('winter > spring & winter > summer & winter > autumn'))


# а таких, собственно, и нет =)

,winter,spring,summer,autumn
Peter Jackson,7.0,8.0,8.0,7.0
Steven Soderbergh,6.0,11.0,11.0,10.0
Clint Eastwood,6.0,12.0,10.0,8.0
Adam Shankman,4.0,6.0,6.0,8.0
Shawn Levy,4.0,8.0,9.0,7.0


,winter,spring,summer,autumn


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [46]:
companies_list = pd.DataFrame(data.production_companies.str.split('|').explode())

# добавляем колонку с длиной названия фильма
companies_list['name_length'] = data.original_title.str.len()

# находим значение средней длины
answers['24'] = companies_list.\
                groupby('production_companies').\
                name_length.mean().\
                idxmax()

display(answers['24'])

'Four By Two Productions'

#### *Вариант 2*

In [47]:
# добавляем колонку с длиной названия фильма
companies['name_length'] = data.original_title.str.len()

# находим значение средней длины
answers['24'] = companies.\
                melt(id_vars = 'name_length', value_name = 'company').\
                groupby('company').\
                name_length.mean().\
                idxmax()

# возвращаем DF к исходному виду
companies = companies.drop('name_length', axis = 1)

display(answers['24'])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [48]:
companies_list = pd.DataFrame(data.production_companies.str.split('|').explode())

# добавляем колонку с длиной описания фильма
companies_list['overview_length'] = data.overview.str.count('\s')+1

# находим значение средней длины
answers['25'] = companies_list.\
                groupby('production_companies').\
                overview_length.mean().\
                idxmax()

display(answers['25'])

'Midnight Picture Show'

#### *Вариант 2*

In [49]:
# добавляем колонку с длиной описания фильма
companies['overview_length'] = data.overview.str.count('\s')+1

# находим значение средней длины
s = companies.\
    melt(id_vars = 'overview_length', value_name = 'company').\
    groupby('company').\
    overview_length.mean().\
    idxmax()

# возвращаем DF к исходному виду
companies = companies.drop('overview_length', axis = 1)

display(s)

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [50]:
answers['26'] = data[data.vote_average > data.vote_average.quantile(q = 0.99)].original_title
display(answers['26'])

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

#### *Вариант 2*

In [51]:
data.sort_values(by='vote_average').tail(int(data.original_title.count()/100)).original_title

155                          The Theory of Everything
863             The Lord of the Rings: The Two Towers
1688                              There Will Be Blood
283                                        Mr. Nobody
1191                                 12 Years a Slave
872                                       The Pianist
128                                         Gone Girl
1800                                          Memento
119                           Guardians of the Galaxy
370                                         Inception
1183                          The Wolf of Wall Street
138                          The Grand Budapest Hotel
1081    The Lord of the Rings: The Return of the King
34                                               Room
118                                      Interstellar
9                                          Inside Out
125                                The Imitation Game
599                                   The Dark Knight
Name: original_title, dtype:

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [52]:
'''сортируем актеров в алфавитном порядке по каждому фильму
для того, чтобы избежать пар с "реверсивным" порядком имен'''
sorted_cast_list = pd.DataFrame(data.cast.str.split('|').apply(lambda x: sorted(x)))

# генерируем комбинации (пары) актеров
casts_combinations = sorted_cast_list.cast.apply(lambda r: list(combinations(r, 2)))

# подсчитывает количество пар
answers['27'] = pd.DataFrame(list(casts_combinations)).\
                melt(value_name = 'cast').\
                cast.value_counts().\
                head()

display(answers['27'])

(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
Name: cast, dtype: int64

#### *Вариант 2 (неверный)*

In [53]:
# без сортировки
casts_combinations = casts.apply(lambda r: list(combinations(r, 2)), axis = 1)

pd.DataFrame(list(casts_combinations)).\
            melt(value_name = 'cast').\
            cast.value_counts().\
            head()

(Daniel Radcliffe, Rupert Grint)    8
(Daniel Radcliffe, Emma Watson)     8
(Rupert Grint, Emma Watson)         7
(None, None)                        6
(Ben Stiller, Owen Wilson)          6
Name: cast, dtype: int64

# Submission

In [54]:
answers

{'1': 723    Pirates of the Caribbean: On Stranger Tides
 Name: original_title, dtype: object,
 '2': 1157    Gods and Generals
 Name: original_title, dtype: object,
 '3': 768    Winnie the Pooh
 Name: original_title, dtype: object,
 '4': 109.7,
 '5': 107.0,
 '6': 239    Avatar
 Name: original_title, dtype: object,
 '7': 1245    The Lone Ranger
 Name: original_title, dtype: object,
 '8': 1478,
 '9': 599    The Dark Knight
 Name: original_title, dtype: object,
 '10': 1245    The Lone Ranger
 Name: original_title, dtype: object,
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 925    K-19: The Widowmaker
 Name: original_title, dtype: object,
 '19': sum    2015
 dtype: int64,
 '20': sum    2014
 dtype: int64,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 9                                          Inside Out
 34    

In [55]:
print('Выполнено:', len(answers), 'заданий из 27')

Выполнено: 27 заданий из 27
